<a id="1"></a>
# <a id='toc1_'></a><div style="text-align:center; border-radius:15px 50px; padding:15px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#2a6199; overflow:hidden"><b> 🧠  TP1 - Quantification des modèles IA </b>

Ce notebook démontre comment appliquer des techniques de **quantification** aux modèles IA pour améliorer leurs performances et réduire leur taille. Vous apprendrez :
- Les concepts clés de la **quantification des modèles**.
- L'application de la quantification sur un exemple simple.
- La quantification d'un modèle d'image plus complexe.



  ---
  ### <a id='toc1_1_1_'></a>[**Table des matières**](#toc0_)

- [📘 1. Introduction à la quantification](#toc1_)    
  - [1.1 Types de quantification](#toc3_)    
  - [1.2 Les compromis à prendre en compte](#toc4_)    
- [🔍 2. Échauffement : Bases de la quantification](#toc5_)    
  - [2.1 Création d'une série de valeurs](#toc6_)    
  - [2.2 Quantification Asymétrique](#toc7_)    
  - [2.3 Quantification Symétrique](#toc8_)    
  - [2.4 Méthode de quantification par Percentile](#toc9_)    
  - [2.5 Comparaison : Symetrique, Asymetrique, Min-Max , Percentile](#toc10_)    
  - [2.6 Analyse de l'impact du nombre des bits sur l'erreur](#toc11_)    
- [🖼️ 3. Application aux Modèles Complexes](#toc12_)    
  - [3.1 Quantification aprés entrainement (Post Training Quantization : PTQ)](#toc13_)    
  - [3.2 Quantification pendant l'entraînement (Quantization Aware Training : QAT)](#toc22_)    
 
- [📊 4. Évaluation et comparaison](#toc25_)    
  - [4.1 Comparaison des tailles et des performances des modèles](#toc26_)    
  - [4.2 Conclusion](#toc29_)      
- [🎁 Bonus](#toc32_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->
  ---

# <a id='toc2_'></a>[<div style="text-align:center; border-radius:8px; padding:8px; color:white; margin:10px 0; font-size:100%; font-family:Arial, sans-serif; background-color:#1E90FF;"><b>📘 1. Introduction à la quantification</b></div>](#toc2_)
# 

En traitement du signal numérique, la quantification fait généralement référence à la conversion d’un signal continu, comme la lumière ou le son, en un ensemble de valeurs numériques discrètes. Dans le domaine de l’apprentissage profond, la quantification désigne généralement la conversion de nombres en virgule flottante signée en précision simple (float32) en un format numérique de précision inférieure, tel qu’un entier non signé sur 8 bits (uint8). Elle permet :

- Une **réduction de la taille des modèles**.
- Une **accélération des inférences**.
- Un **support pour le matériel contraint** (par exemple : IoT, téléphones mobiles).

# <a id='toc3_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:70%; font-family:Arial, sans-serif; background-color:#f17c12;"><b>1.1 Types de quantification</b></div>](#toc3_)


<img src="https://github.com/bouzid-s/IASSC/blob/main/sources/types_quant.png?raw=true" alt="typesQuant" width="700">

# <a id='toc4_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:70%; font-family:Arial, sans-serif; background-color:#f17c12;"><b>1.2 Les compromis à prendre en compte</b></div>](#toc4_)



1. **Taille du modèle :**
   - Réduction typique de 75% pour des poids convertis de Float32 à INT8.

2. **Précision :**
   - La quantification peut entraîner une perte de précision due à l’arrondi des valeurs.
   - **Symétrique :** Plus adaptée pour les poids distribués autour de zéro.
   - **Asymétrique :** Utile pour préserver la précision des activations positives.

3. **Vitesse :**
   - Les modèles quantifiés sont plus rapides en inférence, surtout sur des processeurs spécialisés (comme les DSP ou TPU).

4. **Compatibilité matérielle :**
   - Certains matériels favorisent la quantification asymétrique pour les activations.

---

# <a id='toc5_'></a>[<div style="text-align:center; border-radius:8px; padding:8px; color:white; margin:10px 0; font-size:100%; font-family:Arial, sans-serif; background-color:#1E90FF;"><b>🔍 2. Échauffement : Bases de la quantification</b></div>](#toc5_)
# 

  ---
  ### <a id='toc1_1_1_'></a>[**Table des matières**](#toc0_)

- [1. Introduction à la quantification](#toc1_)    
  - [1.1 Types de quantification](#toc3_)    
  - [1.2 Les compromis à prendre en compte](#toc4_)    
- [🔍 2. Échauffement : Bases de la quantification](#toc5_)    
  - [2.1 Création d'une série de valeurs](#toc6_)    
  - [2.2 Quantification Asymétrique](#toc7_)    
  - [2.3 Quantification Symétrique](#toc8_)    
  - [2.4 Méthode de quantification par Percentile](#toc9_)    
  - [2.5 Comparaison : Symetrique, Asymetrique, Min-Max , Percentile](#toc10_)    
  - [2.6 Analyse de l'impact du nombre des bits sur l'erreur](#toc11_)    
- [🖼️ 3. Application aux Modèles Complexes](#toc12_)    
  - [3.1 Quantification aprés entrainement (Post Training Quantization PTQ)](#toc13_)    
  - [3.2 Quantification pendant l'entraînement (Quantization Aware Training)](#toc22_)    
 
- [📊 4. Évaluation et comparaison](#toc25_)    
  - [4.1 Comparaison des tailles et des performances des modèles](#toc26_)    
  - [4.2 Conclusion](#toc29_)      
- [Bonus](#toc32_)    

<!-- vscode-jupyter-toc-config
   numbering=false
   anchor=true
   flat=false
   minLevel=1
   maxLevel=6
   /vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->
---


Développer un programme en Python qui génère aléatoirement une série de valeurs synthétiques et applique des méthodes de quantification **symétrique** et **asymétrique**. Ce programme devra analyser les impacts de la quantification sur les données et évaluer les erreurs associées.

---
### Étapes à suivre :

1. **Création d'une série de valeurs :**
   - Générer un vecteur aléatoire ou utiliser un ensemble fixe de valeurs.

2. **Quantification :**
   - Implémenter les méthodes de quantification symétrique et asymétrique.
   - Tester pour différents intervalles $[\alpha, \beta]$.

1. **Déquantification :**
   - Revenir aux valeurs approximatives dans l'échelle initiale.

2. **Calcul de l'erreur de quantification :**
   - Calculer l'erreur quadratique moyenne (MSE) pour chaque méthode de quantification.

---

### Résultats attendus :
- Afficher les valeurs originales, quantifiées, et déquantifiées.
- Visualiser les erreurs de quantification sous forme graphique.
- Calculer et comparer les erreurs entre les deux méthodes.
- L'erreur sera calculée en utilisant la formule suivante :
$  MSE = \frac{1}{n} \sum_{i=1}^n \left( x_{\text{original}, i} - x_{\text{déquantifié}, i} \right)^2$

### Utilitaires

In [ ]:
import matplotlib.pyplot as plt
import sys
import numpy as np
import tensorflow as tf

print("Tensorflow", tf.__version__)
print("Python", sys.version)

# Suppress scientific notation
np.set_printoptions(suppress=True)


def calculate_errors(original, dequantized):
    mse = np.mean((original - dequantized)**2)
    mae = np.mean(np.abs(original - dequantized))
    return mse, mae

def plot_original_vs_quantized(x_original, x_quant, title="Original vs Quantifié ", c='gray'):
    # Création des sous-graphiques
    plt.figure(figsize=(12, 6))
    # Premier sous-graphe : Original vs Quantifié
    plt.scatter(x_original, x_quant, color='orange', alpha=0.7, label="Quantifié ")
    plt.plot(x_original, x_original, color='red', linestyle='--', label="Ligne parfaite (y=x)")
    plt.title(title, fontsize=14, fontweight='bold')
    plt.ylabel("Valeurs Quantifiées", fontsize=12)
    plt.grid(linestyle='--', alpha=0.6)
    plt.legend(fontsize=10, frameon=True, loc='upper left')
    # Ajustement des espaces entre les sous-graphiques
    plt.tight_layout()
    # Affichage du graphique
    plt.show()


def plot_histograms(x_original, x_dequant_asym, title="Original vs DeQuantifié ", c='gray'):
    plt.figure(figsize=(12, 6))
    plt.hist(x_original, bins=50, alpha=0.7, label="Original",color='orange', edgecolor='black', linewidth=0.5)
    plt.hist(x_dequant_asym, bins=50, alpha=0.5, label="Déquantifié",  color=c, edgecolor='black', linewidth=0.5)
    plt.title(title, fontsize=14, fontweight='bold')
    plt.ylabel("Fréquence", fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.legend(frameon=True, fontsize=10, loc='upper right')

def plot_error(x_original, x_dequant, title, c='gray'):
    plt.figure(figsize=(12, 6))
    # Calcul des erreurs absolues pour chaque type de quantification
    error = np.abs(x_original - x_dequant)

    # Calcul des moyennes d'erreur
    mean_error = np.mean(error)
        # Lignes horizontales pour les moyennes
    # Premier sous-graphe : Erreur pour la quantification asymétrique
    plt.plot(error, label="Erreur ("+title+")", color=c, linestyle='-', marker='o', alpha=0.7)
    plt.title("Erreur de Quantification : "+title, fontsize=14, fontweight='bold')
    plt.ylabel("Erreur absolue", fontsize=12)
    plt.axhline(mean_error, color='red', linestyle='--', label=f"Moyenne : {mean_error:.2f}")
    plt.legend(fontsize=10, frameon=True, loc='upper right')
    plt.grid(linestyle='--', alpha=0.6)
    # Ajout des labels et du titre
    plt.xlabel("Index des valeurs", fontsize=12)
    # Ajustement des espaces entre les sous-graphiques
    plt.tight_layout()
    # Affichage du graphique
    plt.show()


# <a id='toc6_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:70%; font-family:Arial, sans-serif; background-color:#f17c12;"><b> 2.1 Création d'une série de valeurs </b></div>](#toc6_)


In [ ]:
# tap your code here ...


# <a id='toc7_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:70%; font-family:Arial, sans-serif; background-color:#f17c12;"><b> 2.2 Quantification Asymétrique </b></div>](#toc7_)

##### Quantification asymétrique :
- Convertit une série de nombres en virgule flottante d’un intervalle $[\beta, \alpha]$ vers l’intervalle $[0, 2^n - 1]$ (par exemple, avec 8 bits : intervalle $[0, 255]$).

**Formules associées :**
- Quantification :
  
  $  x_q = \text{clamp}\left(\left\lfloor \frac{x_f}{s} \right\rfloor + z; 0, 2^n - 1\right), \quad s = \frac{\alpha - \beta}{2^n - 1}, \quad z = \left\lfloor -1 \times \frac{\beta}{s} \right\rfloor $

- Déquantification :  

  $  x_f = s \left( x_q - z \right)$
  

**Illustration :**

<img src="https://github.com/bouzid-s/IASSC/blob/main/sources/asym.png?raw=true" alt="Quantification asymétrique" width="700">


In [3]:
# tap your code here ...


# <a id='toc8_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:70%; font-family:Arial, sans-serif; background-color:#f17c12;"><b> 2.3 Quantification Symétrique </b></div>](#toc8_)



##### Quantification symétrique :
- Convertit une série de nombres en virgule flottante dans l’intervalle $[-\alpha, \alpha]$ vers un intervalle symétrique en entiers $[-(2^{n-1} - 1), 2^{n-1} - 1]$. 
- Par exemple, avec 8 bits, cet intervalle est $[-127, 127]$.

**Formules associées :**
- Quantification :  
- 
  $  x_q = \text{clamp}\left(\left\lfloor \frac{x_f}{s} \right\rfloor; -(2^{n-1} - 1), 2^{n-1} - 1\right), \quad s = \frac{\text{abs}(\alpha)}{2^{n-1} - 1}$

- Déquantification :  

  $  x_f = s \left( x_q \right)$

**Illustration :**

<img src="https://github.com/bouzid-s/IASSC/blob/main/sources/sym.png?raw=true" alt="Quantification symétrique" width="700">

In [5]:
# tap your code here ...

# <a id='toc9_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:70%; font-family:Arial, sans-serif; background-color:#f17c12;"><b> 2.4 Méthode de quantification par Percentile</b></div>](#toc9_)


**Principe :**
La méthode **Percentile** utilise des percentiles ($P_1$ et $P_{99}$) pour définir l'intervalle de quantification, en ignorant les valeurs aberrantes dans les queues de la distribution.

**Étapes d'implémentation :**
1. **Calcul des percentiles :**
   - $P_1 = \text{np.percentile(x, 1)}$
   - $P_{99} = \text{np.percentile(x, 99)}$


**Consigne :** 
Comparez les performances de deux méthodes de quantification, **Min-Max** et **Percentile**, en termes de précision et de robustesse.

**Objectif :** 

Identifier dans quels contextes chaque méthode est la plus adaptée.

In [ ]:
# tap your code here ...


# <a id='toc10_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:70%; font-family:Arial, sans-serif; background-color:#f17c12;"><b> 2.5 Comparaison : Symetrique, Asymetrique, Min-Max , Percentile </b></div>](#toc10_)



In [ ]:
# tap your code here ...


# <a id='toc11_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:70%; font-family:Arial, sans-serif; background-color:#f17c12;"><b> 2.6 Analyse de l'impact du nombre des bits sur l'erreur </b></div>](#toc11_)

L'erreur dépend de l'intervale des valeurs, de leur distribution et du nombre de bits utilisés pour la quantification. Comme les distributions des poids et des entrées d'un réseau de neurones ne sont pas contrôlées, nous analysons ici l'impact de la largeur des bits sur l'erreur. 

#### Consignes :
Analysez l'impact de la largeur des bits sur l'erreur absolue moyenne en développant un programme qui :

- **Quantifie** et **déquantifie** les données pour différentes largeurs de bits `bitwidths = [i for i in range(2, 16)]
`.
- **Calcule** l'erreur absolue moyenne pour chaque largeur de bits.
- **Visualise** la relation entre la largeur des bits et l'erreur à l'aide d'un graphique.

In [ ]:
# tap your code here ...

# <a id='toc12_'></a>[<div style="text-align:left; border-radius:8px; padding:8px; color:white; margin:10px 0; font-size:70%; font-family:Arial, sans-serif; background-color:#1E90FF;"><b> 🖼️ 3. Application aux Modèles Complexes </b></div>](#toc12_)
# 

Dans cette section, vous appliquerez des techniques de quantification sur des modèles complexes, comme ceux utilisés pour la classification d'images (par exemple, MNIST). Vous explorerez deux approches principales de quantification : **la quantification après entraînement (PTQ)**, qui est réalisée sur un modèle déjà entraîné, et **la quantification durant l'entraînement (QAT)**, qui permet d'incorporer la quantification directement dans le processus d'entraînement. Vous analyserez l'impact de ces deux méthodes sur la taille du modèle et ses performances.


  ---
  ### <a id='toc1_1_1_'></a>[**Table des matières**](#toc0_)

- [1. Introduction à la quantification](#toc1_)    
  - [1.1 Types de quantification](#toc3_)    
  - [1.2 Les compromis à prendre en compte](#toc4_)    
- [🔍 2. Échauffement : Bases de la quantification](#toc5_)    
  - [2.1 Création d'une série de valeurs](#toc6_)    
  - [2.2 Quantification Asymétrique](#toc7_)    
  - [2.3 Quantification Symétrique](#toc8_)    
  - [2.4 Méthode de quantification par Percentile](#toc9_)    
  - [2.5 Comparaison : Symetrique, Asymetrique, Min-Max , Percentile](#toc10_)    
  - [2.6 Analyse de l'impact du nombre des bits sur l'erreur](#toc11_)    
- [🖼️ 3. Application aux Modèles Complexes](#toc12_)    
  - [3.1 Quantification aprés entrainement (Post Training Quantization PTQ)](#toc13_)    
  - [3.2 Quantification pendant l'entraînement (Quantization Aware Training)](#toc22_)    
 
- [📊 4. Évaluation et comparaison](#toc25_)    
  - [4.1 Comparaison des tailles et des performances des modèles](#toc26_)    
  - [4.2 Conclusion](#toc29_)      
- [Bonus](#toc32_)    

<!-- vscode-jupyter-toc-config
    numbering=false
    anchor=true
    flat=false
    minLevel=1
    maxLevel=6
    /vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->
  ---

# <a id='toc13_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:70%; font-family:Arial, sans-serif; background-color:#f17c12;"><b> 3.1 Quantification aprés entrainement (Post Training Quantization : PTQ) </b></div>](#toc13_)



La quantification après entraînement (PTQ) consiste à convertir un modèle déjà entraîné en un format plus compact. Nous évaluerons ici l’impact de cette méthode sur la taille et la précision du modèle.


# <a id='toc14_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:50%; font-family:Arial, sans-serif; background-color:#115175;"><b> a) Définition d’un modèle de classification sur MNIST </b></div>](#toc14_)



**Consigne :**  

Développez un programme en Python pour créer et compiler un modèle de réseau de neurones simple en utilisant TensorFlow/Keras. 

**Instructions**
- Implémentez un modèle en utilisant `tf.keras.Sequential`.
- Intégrez les couches suivantes :
  - **Flatten** : Cette couche doit recevoir une entrée de forme `(28, 28)` et l’aplatir en un vecteur unidimensionnel.
  - **Dense** : Ajoutez une couche dense avec 128 unités et la fonction d’activation **ReLU**.
  - **Dropout** : Insérez une couche de régularisation avec un taux de dropout de 0.2 pour réduire le surapprentissage.
  - **Dense** : Terminez par une couche dense avec 10 unités et la fonction d’activation **softmax** pour effectuer la classification.
- Compilez le modèle en spécifiant :
  - Optimiseur : `Adam` avec un taux d'apprentissage défini par la constante `LEARNING_RATE`.
  - Fonction de perte : `SparseCategoricalCrossentropy`.
  - Métrique : `accuracy`.
- Retournez le modèle à l’aide d’une fonction `create_model()`.


In [ ]:
# !pip install tensorflow_datasets
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_model_optimization as tfmot
import numpy as np
import os
from pathlib import Path


#--------------------PARAMs--------------------#
# Constants
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 0.001
SEED = 42
MODEL_DIR = "./my_models/"


# Ensure reproducibility
tf.random.set_seed(SEED)

# Load the MNIST dataset
(ds_train, ds_validation, ds_test), ds_info = tfds.load(
    "mnist",
    split=['train[:85%]', 'train[85%:95%]', 'test'],
    as_supervised=True,
    with_info=True,
)

# Normalize the dataset
def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255.0, label

ds_train = ds_train.map(normalize_img).batch(BATCH_SIZE).shuffle(10000, seed = SEED)
ds_validation = ds_validation.map(normalize_img).batch(BATCH_SIZE)
ds_test = ds_test.map(normalize_img).batch(1)  # Batch size set to 1 for testing

# Define the model
def create_model():
    """Create and compile a simple feedforward neural network."""

    return model

model = create_model()
# Print model summary
print("Model Summary:")
model.summary()



# <a id='toc15_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:50%; font-family:Arial, sans-serif; background-color:#115175;"><b> b)  Entraînement d’un modèle de classification sur MNIST</b></div>](#toc15_)

**Entraînement du modèle** :
   - Utilisez la méthode `model.fit()` pour entraîner le modèle sur les données.
   - Spécifiez les paramètres suivants :
     - `ds_train` comme ensemble d’entraînement.
     - `ds_validation` comme ensemble de validation avec le paramètre `validation_data`.
     - `EPOCHS` comme nombre d’époques d’entraînement.

**Sauvegarde de l’historique** :
   - Assurez-vous que l'objet `history` contient les informations de l’entraînement pour pouvoir analyser les métriques d’entraînement et de validation ultérieurement.


In [ ]:
# Train the model





# Visualize training metrics
def plot_training_history(history):
    """Plot training and validation loss and accuracy."""
    plt.figure(figsize=(12, 6))
    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(alpha=0.6)

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(alpha=0.6)

    plt.tight_layout()
    plt.show()

plot_training_history(history)


# <a id='toc16_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:50%; font-family:Arial, sans-serif; background-color:#115175;"><b> c) Afficher les poids et la taille du modèle avant la quantification </b></div>](#toc16_)


**Consigne :**  
1. Affichez les poids des différentes couches de votre modèle à l'aide de la méthode `get_weights()`.  
2. Calculez la taille des poids pour chaque couche en mégaoctets (MB) et affichez ces informations de manière structurée.  
3. Calculez la taille totale du modèle sauvegardé en utilisant `os.path.getsize()` pour évaluer la mémoire occupée avant la quantification.  

  
Cette étape vous permettra de comprendre la répartition de la mémoire utilisée par un modèle non quantifié et de comparer ces résultats avec ceux du modèle quantifié.


In [ ]:

# Print weights
print("\nModel Weights:")
for layer in model.layers:
    weights = layer.get_weights()
    if weights:  # Skip layers with no weights (e.g., activations, flatten layers)
        print(f"Layer: {layer.name}")
        for i, weight in enumerate(weights):
            print(f"  Weight {i}: shape = {weight.shape}, size = {weight.nbytes / 1e3:.2f} KB")
            print(weight )

# <a id='toc17_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:50%; font-family:Arial, sans-serif; background-color:#115175;"><b> d) Enregistrer le modèle dans votre espace de travail au format TensorFlow </b></div>](#toc17_)


**Consigne :**  

1. **Création des répertoires** :
   - Créez un répertoire pour stocker le modèle sauvegardé.
   - Assurez-vous que le répertoire existe avant de tenter d'y écrire.

2. **Sauvegarde du modèle** :
   - Sauvegardez le modèle dans le répertoire spécifié en utilisant `model.export()` ou une méthode similaire.

3. **Évaluation du modèle** :
   - Évaluez les performances du modèle sur l’ensemble de test (`ds_test`) en utilisant `model.evaluate()`.
   - Affichez les valeurs de la perte (`loss`) et de la précision (`accuracy`).

4. **Calcul de la taille du modèle** :
   - Calculez la taille totale du modèle sauvegardé (en MB) en parcourant tous les fichiers du répertoire de sauvegarde.
   - Affichez la taille du modèle en utilisant un format lisible.


Cette étape prépare le modèle pour des étapes futures notamment la quantification.


Cette étape prépare le modèle pour des étapes futures notamment la quantification.


In [ ]:
# tap your code here ...

# <a id='toc18_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:50%; font-family:Arial, sans-serif; background-color:#115175;"><b> e) Fonctions Utilitaires </b></div>](#toc18_)

#### **i. `print_stats_model`**

**Objectif :**  
Analyser les statistiques et les détails des tenseurs d’un modèle TFLite quantifié.

**Arguments :**  
- `model` : Chemin du fichier TFLite.  
- `m_name` : Nom du modèle pour l’affichage.  
- `show_weights` : Affiche les valeurs des poids si activé.

**Exemple d’utilisation :**  
```python
print_stats_model("mnist_model_fp16.tflite", m_name="MNIST Quantized Model", show_weights=False)

In [ ]:
def print_stats_model(model, m_name="model", show_weights=False):
    """
    Prints statistics and details about a quantized TFLite model.

    Args:
        model (str): Path to the TFLite model file.
        m_name (str): Model name for display.
        show_weights (bool): Whether to display weight values. Defaults to False.
    """
    try:
        # Print model information
        print(f"\nStats for: {m_name}")
        
        # Evaluate model size
        model_size = os.path.getsize(model)
        print(f"Model size: {model_size / 1e6:.2f} MB")

        # Load the TFLite model
        interpreter = tf.lite.Interpreter(model_path=model)
        interpreter.allocate_tensors()

        # Analyze and print tensor details
        tensor_details = interpreter.get_tensor_details()
        total_weight_size = 0
        print("\nQuantized Model Details:")
        
        for tensor in tensor_details:
            print(f"Tensor Name: {tensor['name']}")
            print(f"  Shape: {tensor['shape']}")
            print(f"  Data Type: {tensor['dtype']}")
            print(f"  Quantization Parameters: {tensor['quantization']}")
            
            # Get the actual tensor data
            weights = interpreter.tensor(tensor['index'])()
            weight_size = weights.nbytes / 1e3  # Convert to KB
            total_weight_size += weight_size
            print(f"  Size: {weight_size:.2f} KB")
            
            if show_weights:
                print(f"  Weights Values: {weights}")
            
            print("-" * 50)

        # Print total weight size
        print(f"\nTotal Weights Size: {total_weight_size / 1e3:.2f} MB")

    except Exception as e:
        print(f"Error processing the model: {e}")

#### **ii. `evaluate_quantized_model`**

**Objectif :**  
Tester les performances d’un modèle TFLite quantifié en évaluant sa précision sur un dataset donné.

 **Arguments :**
- `model_path` : Chemin vers le fichier TFLite contenant le modèle quantifié.
- `dataset` : Dataset TensorFlow utilisé pour l’évaluation, comme `ds_test`.
- `m_name` : Nom du modèle pour l’affichage (par défaut : `"Quantized Model"`).

 **Fonctionnalités :**
1. Charge le modèle TFLite à partir du chemin fourni.
2. Affiche les détails des entrées du modèle, comme la forme (`shape`) et le type (`dtype`).
3. Boucle sur le dataset d’évaluation :
   - Vérifie et ajuste la forme des entrées si nécessaire.
   - Effectue des inférences en utilisant le modèle TFLite.
   - Compare les prédictions aux labels réels pour calculer la précision.
4. Affiche la précision finale (accuracy) du modèle.

 **Exemple d’utilisation :**
 
```python
evaluate_quantized_model("mnist_model_fp16.tflite", ds_test, m_name="MNIST Float16 Quantized Model")

In [ ]:
def evaluate_quantized_model(model_path, dataset, m_name="Quantized Model"):
    """
    Evaluates the performance of a quantized TFLite model on a given dataset.

    Args:
        model_path (str): Path to the TFLite model file.
        dataset (tf.data.Dataset): A TensorFlow dataset for evaluation.
        m_name (str): Name of the model for display.
    """
    try:
        if not os.path.exists(model_path):
            print(f"Error: Model file '{model_path}' does not exist.")
            return

        print(f"\nEvaluating {m_name}")

        # Load the TFLite model
        interpreter = tf.lite.Interpreter(model_path=model_path)
        interpreter.allocate_tensors()

        # Get input and output details
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        # Check input shape and type
        input_shape = input_details[0]['shape']
        input_dtype = input_details[0]['dtype']

        print(f"Model Input Shape: {input_shape}")
        print(f"Model Input Data Type: {input_dtype}")

        # Initialize counters for accuracy calculation
        # Test the model
        correct_predictions = 0
        total_predictions = 0

        for images, labels in ds_test:
            # Remove the batch dimension for single image inference
            input_data = tf.squeeze(images, axis=-1).numpy()  # From [1, 28, 28, 1] to [28, 28, 1]

             # Ensure input_data matches the expected dtype
            #input_data = input_data.astype(input_details[0]['dtype'])
            # Adjust input data based on the model's expected dtype
            if input_dtype == np.uint8:
                input_data = (input_data * 255).astype(np.uint8)  # Scale to [0, 255]
            elif input_dtype == np.float32:
                input_data = input_data.astype(np.float32)  # Keep in [0, 1] range
            else:
                raise ValueError(f"Unsupported input data type: {input_dtype}")

            # Set the tensor to the input data
            interpreter.set_tensor(input_details[0]['index'], input_data)

            # Run inference
            interpreter.invoke()

            # Get the prediction
            output_data = interpreter.get_tensor(output_details[0]['index'])
            predicted_label = np.argmax(output_data)

            # Compare with the true label
            if predicted_label == labels.numpy():
                correct_predictions += 1
            total_predictions += 1

        # Calculate accuracy
        accuracy = correct_predictions / total_predictions * 100
        print(f"Accuracy of the quantized {model_path}: {accuracy:.2f}%")

    except Exception as e:
        print(f"Error evaluating the model: {e}")

# <a id='toc20_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:50%; font-family:Arial, sans-serif; background-color:#115175;"><b> f) Appliquer une réduction flottante : Quantification FP32 → FP16 </b></div>](#toc20_)



 **Objectif :**
 Appliquer une quantification post-entraînement au modèle sauvegardé en convertissant celui-ci en format TFLite avec Float16 et en sauvegardant la version quantifiée. Cela permet d’économiser de l’espace de stockage et d’accélérer l’inférence sur des appareils compatibles avec le calcul FP16.

 **Étapes à suivre :**


1. **Chargement du modèle sauvegardé** :
   - Utilisez `TFLiteConverter.from_saved_model()` pour charger le modèle préalablement sauvegardé depuis le répertoire spécifié.

2. **Application des optimisations** :

3. **Configuration de la quantification Float16** :

4. **Conversion du modèle** :

5. **Sauvegarde du modèle quantifié** :
   - Sauvegardez le modèle Float16 TFLite dans un fichier avec une extension `.tflite`.
   - Assurez-vous que le répertoire cible existe avant de sauvegarder le fichier.

6. **Affichage de confirmation** :
   - Affichez un message confirmant la réussite de la conversion et la localisation du fichier quantifié.


 **Ressources complémentaires :**

Pour en savoir plus sur la quantification flottante avec TensorFlow Lite, consultez cet [article de blog](https://blog.tensorflow.org/2019/08/tensorflow-model-optimization-toolkit_5.html).

In [ ]:
# tap your code here ...


#### **Afficher des statistiques sur le modèle quantifié en FP16**

 **Objectif :**
Analyser le modèle quantifié en FP16 pour obtenir des informations détaillées sur sa structure, ses dimensions, les paramètres de quantification et la taille de ses poids.

 **Étapes à suivre :** utiliser les foncttions utilitaires



In [ ]:
# tap your code here ...

# <a id='toc21_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:50%; font-family:Arial, sans-serif; background-color:#115175;"><b> e) Appliquer une Quantification FP32 --> INT8 et enregistrer le nouveau modèle quantifié</b></div>](#toc21_)


**Objectif :**
Réduire la précision du modèle en passant de FP32 à INT8 pour diminuer sa taille et améliorer ses performances sur des appareils aux ressources limitées.

 **Étapes à suivre :**

1. **Configurer le convertisseur TFLite pour la quantification INT8 :**
   - Charger le modèle au format TensorFlow.
   - Définir une optimisation de type INT8.
   - Spécifier un jeu de données représentatif pour calibrer la quantification.

2. **Effectuer la conversion :**
   Convertir le modèle TensorFlow en un modèle TFLite quantifié en INT8.

3. **Enregistrer le modèle converti :**
   Sauvegarder le modèle quantifié au format `.tflite`.

**Ressources complémentaires :**

Pour en savoir plus sur la quantification entière avec TensorFlow Lite, consultez cet [article de blog](https://blog.tensorflow.org/2019/06/tensorflow-integer-quantization.html?hl=fr).

In [ ]:
# tap your code here ...

#### **Afficher des statistiques sur le modèle quantifié en INT8**

 **Objectif :**
Analyser le modèle quantifié en INT8 pour obtenir des informations détaillées sur sa structure, ses dimensions, les paramètres de quantification et la taille de ses poids.

 **Étapes à suivre :** utiliser les foncttions utilitaires



In [ ]:
# tap your code here ...

# <a id='toc22_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:70%; font-family:Arial, sans-serif; background-color:#f17c12;"><b> 3.2 Quantification pendant l'entraînement (Quantization Aware Training : QAT) </b></div>](#toc22_)


# <a id='toc23_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:50%; font-family:Arial, sans-serif; background-color:#115175;"><b> a) Quantification complète du modèle </b></div>](#toc23_)



**Objectif :**  
Dans cette section, nous appliquons la quantification pendant l'entraînement à un modèle complet. Cela consiste à intégrer des opérations de quantification et de déquantification dans le flux d'entraînement du modèle, afin d'améliorer ses performances en conditions réelles tout en réduisant sa taille.

**Consigne :**  
- Implémentez la quantization aware training (QAT) sur l'intégralité du modèle. 
- Entraînez le modèle sur le dataset MNIST pour démontrer l'efficacité de cette technique.
- Comparez les performances du modèle quantifié avec celles du modèle original.

In [ ]:
# tap your code here ...

# <a id='toc24_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:50%; font-family:Arial, sans-serif; background-color:#115175;"><b> b) Quantification partielle du modèle </b></div>](#toc24_)

**Objectif :**  
Dans cette section, nous appliquons la quantization aware training (QAT) uniquement à certaines couches du modèle. Cela permet de maintenir un compromis entre la réduction de la taille et la conservation de performances élevées.

**Consigne :**  
- Sélectionnez uniquement la couche dense du modèle pour appliquer la quantification.
- Entraînez le modèle sur le dataset MNIST en intégrant cette quantification partielle.
- Comparez les performances du modèle quantifié partiellement avec celles du modèle original et du modèle entièrement quantifié.


In [ ]:
# tap your code here ...

# <a id='toc25_'></a>[<div style="text-align:left; border-radius:8px; padding:8px; color:white; margin:10px 0; font-size:70%; font-family:Arial, sans-serif; background-color:#1E90FF;"><b> 📊 4. Évaluation et comparaison </b></div>](#toc25_)


  ---
  ### <a id='toc1_1_1_'></a>[**Table des matières**](#toc0_)

- [1. Introduction à la quantification](#toc1_)    
  - [1.1 Types de quantification](#toc3_)    
  - [1.2 Les compromis à prendre en compte](#toc4_)    
- [🔍 2. Échauffement : Bases de la quantification](#toc5_)    
  - [2.1 Création d'une série de valeurs](#toc6_)    
  - [2.2 Quantification Asymétrique](#toc7_)    
  - [2.3 Quantification Symétrique](#toc8_)    
  - [2.4 Méthode de quantification par Percentile](#toc9_)    
  - [2.5 Comparaison : Symetrique, Asymetrique, Min-Max , Percentile](#toc10_)    
  - [2.6 Analyse de l'impact du nombre des bits sur l'erreur](#toc11_)    
- [🖼️ 3. Application aux Modèles Complexes](#toc12_)    
  - [3.1 Quantification aprés entrainement (Post Training Quantization PTQ)](#toc13_)    
  - [3.2 Quantification pendant l'entraînement (Quantization Aware Training)](#toc22_)    
 
- [📊 4. Évaluation et comparaison](#toc25_)    
  - [4.1 Comparaison des tailles et des performances des modèles](#toc26_)    
  - [4.2 Conclusion](#toc29_)      
- [Bonus](#toc32_)    

<!-- vscode-jupyter-toc-config
    numbering=false
    anchor=true
    flat=false
    minLevel=1
    maxLevel=6
    /vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->
  ---
  
# <a id='toc26_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:70%; font-family:Arial, sans-serif; background-color:#f17c12;"><b> 4.1 Comparaison des tailles et des performances des modèles </b></div>](#toc26_)


**Objectif :**  
L'objectif de cette section est d'analyser l'impact de la quantification sur la taille et les performances des modèles. Nous allons comparer les modèles Float32 (modèle original), Float16 et INT8 en termes de taille et de précision.

---
# <a id='toc27_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:50%; font-family:Arial, sans-serif; background-color:#115175;"><b> a) Étape 1 : Afficher la taille des modèles </b></div>](#toc27_)


Utilisez la fonction `print_stats_model` pour calculer et afficher les tailles respectives des modèles Float32, Float16 et INT8. Cela permettra d'observer la réduction de taille obtenue grâce à chaque méthode de quantification.



In [ ]:
# tap your code here ...

# <a id='toc28_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:50%; font-family:Arial, sans-serif; background-color:#115175;"><b> b) Étape 2 : Comparer les performances des modèles </b></div>](#toc28_)

#### **Objectif :**
Évaluer et comparer les performances des modèles Float32, Float16 et INT8 sur le jeu de test pour analyser l'impact de la quantification.



In [ ]:
# tap your code here ...


# <a id='toc29_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:70%; font-family:Arial, sans-serif; background-color:#f17c12;"><b> 4.2 Conclusion </b></div>](#toc29_)

# <a id='toc30_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:50%; font-family:Arial, sans-serif; background-color:#115175;"><b> a)  Flottant ou entière </b></div>](#toc30_)

1. **Taille des modèles :**  
   - La quantification Float16 permet une réduction significative de la taille du modèle tout en conservant une précision proche de celle du modèle original Float32.
   - La quantification INT8 offre une réduction de taille encore plus importante, au prix d'une éventuelle perte de précision.

2. **Performances des modèles :**  
   - Le modèle Float16 conserve une précision élevée, rendant cette méthode idéale lorsque les contraintes de mémoire ne sont pas strictes.  
   - Le modèle INT8 peut subir une dégradation plus marquée de la précision, mais reste adapté pour des environnements où les contraintes de stockage et de calcul sont prioritaires (comme les appareils embarqués).

3. **Compromis taille-précision :**  
   - Le choix de la méthode de quantification dépend des besoins spécifiques de votre application :  
     - **Float16 :** Meilleur compromis entre taille et précision.  
     - **INT8 :** Réduction maximale de la taille pour des applications nécessitant une faible empreinte mémoire.


La quantification constitue une technique puissante pour optimiser les modèles tout en répondant aux contraintes matérielles spécifiques. Il est essentiel d’évaluer chaque approche dans le contexte de votre cas d’utilisation pour choisir la méthode optimale.

# <a id='toc31_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:50%; font-family:Arial, sans-serif; background-color:#115175;"><b> b) Intégrale ou partielle </b></div>](#toc31_)


1. **Quantification intégrale :**  
   - La quantification complète d'un modèle (toutes les couches) permet une réduction maximale de la taille et des ressources nécessaires pour l'inférence.  
   - Cependant, elle peut entraîner une dégradation de précision plus importante, en particulier pour les modèles complexes ou les tâches nécessitant une grande sensibilité.

2. **Quantification partielle :**  
   - La quantification partielle (appliquée uniquement à certaines couches, par exemple les couches entièrement connectées ou convolutives) offre un compromis intéressant.  
   - Elle permet de réduire la taille et les besoins en calcul tout en limitant l’impact sur la précision globale du modèle.  
   - Cette approche est particulièrement utile pour des modèles critiques, où certaines couches doivent conserver leur précision d'origine pour garantir des performances optimales.

3. **Choix entre intégrale et partielle :**  
   - Le choix entre une quantification intégrale ou partielle dépend des priorités de votre application :  
     - **Quantification intégrale :** Convient aux environnements contraints en mémoire et en calcul, où une certaine dégradation de précision est acceptable.  
     - **Quantification partielle :** Convient lorsque la précision est une priorité tout en bénéficiant d'une optimisation partielle des ressources.

La quantification partielle est souvent privilégiée pour son équilibre entre réduction des ressources et préservation des performances. Une analyse approfondie du modèle et des besoins spécifiques de l'application est nécessaire pour faire le bon choix.



# <a id='toc32_'></a>[<div style="text-align:center; border-radius:20px; padding:18px; color:white; margin:10px 0; font-size:100%; font-family:Arial, sans-serif; background-color:#1E90FF;"><b>🎁 Bonus</b></div>](#toc0_)
<div style="text-align: center; margin: 40px 0;">
    <a href="https://colab.research.google.com/github/sayakpaul/Adventures-in-TensorFlow-Lite/blob/master/Boundless_TFLite.ipynb" target="_parent">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" style="width: 350px; height: auto;">
    </a>
</div>



  ---
  ### <a id='toc1_1_1_'></a>[**Table des matières**](#toc0_)

- [1. Introduction à la quantification](#toc1_)    
  - [1.1 Types de quantification](#toc3_)    
  - [1.2 Les compromis à prendre en compte](#toc4_)    
- [🔍 2. Échauffement : Bases de la quantification](#toc5_)    
  - [2.1 Création d'une série de valeurs](#toc6_)    
  - [2.2 Quantification Asymétrique](#toc7_)    
  - [2.3 Quantification Symétrique](#toc8_)    
  - [2.4 Méthode de quantification par Percentile](#toc9_)    
  - [2.5 Comparaison : Symetrique, Asymetrique, Min-Max , Percentile](#toc10_)    
  - [2.6 Analyse de l'impact du nombre des bits sur l'erreur](#toc11_)    
- [🖼️ 3. Application aux Modèles Complexes](#toc12_)    
  - [3.1 Quantification aprés entrainement (Post Training Quantization PTQ)](#toc13_)    
  - [3.2 Quantification pendant l'entraînement (Quantization Aware Training)](#toc22_)    
 
- [📊 4. Évaluation et comparaison](#toc25_)    
  - [4.1 Comparaison des tailles et des performances des modèles](#toc26_)    
  - [4.2 Conclusion](#toc29_)      
- [Bonus](#toc32_)    

<!-- vscode-jupyter-toc-config
    numbering=false
    anchor=true
    flat=false
    minLevel=1
    maxLevel=6
    /vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->
  ---